## import

In [1]:
import os
import random
import time
import json
import warnings 
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from utils import *
import cv2
from sklearn.model_selection import StratifiedKFold

import numpy as np
import pandas as pd
from tqdm.auto import tqdm

# Pretrained Model
import segmentation_models_pytorch as smp

# torchvision Models
import torchvision
from torchvision import models
from torchvision.models.segmentation.deeplabv3 import DeepLabHead

# 전처리를 위한 라이브러리
from torch.utils.data.sampler import WeightedRandomSampler
from pycocotools.coco import COCO
import torchvision
import torchvision.transforms as transforms

import albumentations as A
from albumentations.pytorch import ToTensorV2

# 시각화를 위한 라이브러리
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

plt.rcParams['axes.grid'] = False

print('pytorch version: {}'.format(torch.__version__))
print('GPU 사용 가능 여부: {}'.format(torch.cuda.is_available()))

print(torch.cuda.get_device_name(0))
print(torch.cuda.device_count())

device = "cuda" if torch.cuda.is_available() else "cpu"   # GPU 사용 가능 여부에 따라 device 정보 저장
%matplotlib inline

pytorch version: 1.5.0+cu101
GPU 사용 가능 여부: True
Tesla V100-PCIE-32GB
1


## 하이퍼파라미터 세팅 및 seed 고정

In [2]:
batch_size = 20  # Mini-batch size
num_epochs = 40
learning_rate = 1e-4

In [3]:
# seed 고정
random_seed = 42
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
# torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

## 학습 데이터 EDA

In [4]:
dataset_path = '../input/data'
anns_file_path = dataset_path + '/' + 'train.json'

# Read annotations
with open(anns_file_path, 'r') as f:
    dataset = json.loads(f.read())

categories = dataset['categories']
anns = dataset['annotations']
imgs = dataset['images']
nr_cats = len(categories)
nr_annotations = len(anns)
nr_images = len(imgs)

# Load categories and super categories
cat_names = []
super_cat_names = []
super_cat_ids = {}
super_cat_last_name = ''
nr_super_cats = 0
for cat_it in categories:
    cat_names.append(cat_it['name'])
    super_cat_name = cat_it['supercategory']
    # Adding new supercat
    if super_cat_name != super_cat_last_name:
        super_cat_names.append(super_cat_name)
        super_cat_ids[super_cat_name] = nr_super_cats
        super_cat_last_name = super_cat_name
        nr_super_cats += 1

print('Number of super categories:', nr_super_cats)
print('Number of categories:', nr_cats)
print('Number of annotations:', nr_annotations)
print('Number of images:', nr_images)

Number of super categories: 11
Number of categories: 11
Number of annotations: 21116
Number of images: 2617


In [5]:
# Count annotations
cat_histogram = np.zeros(nr_cats,dtype=int)
for ann in anns:
    cat_histogram[ann['category_id']] += 1

# Convert to DataFrame
df = pd.DataFrame({'Categories': cat_names, 'Number of annotations': cat_histogram})
df = df.sort_values('Number of annotations', 0, False)

## 데이터 전처리 함수 정의 (Dataset)

In [6]:
# category labeling 
sorted_temp_df = df.sort_index()

# background = 0 에 해당되는 label 추가 후 기존들을 모두 label + 1 로 설정
sorted_df = pd.DataFrame(["Backgroud"], columns = ["Categories"])
sorted_df = sorted_df.append(sorted_temp_df, ignore_index=True)

category_names = list(sorted_df.Categories)

def get_classname(classID, cats):
    for i in range(len(cats)):
        if cats[i]['id']==classID:
            return cats[i]['name']
    return "None"

class CustomDataLoader(Dataset):
    """COCO format"""
    def __init__(self, data_dir, mode = 'train', transform = None):
        super().__init__()
        self.mode = mode
        self.transform = transform
        self.coco = COCO(data_dir)
        self.mask_image=[]
        self.mask_label=[]

        if (self.mode in ('train', 'val')):
            print(f'{self.mode} : Preprocessing...')
            for index in tqdm(range(len(self.coco.getImgIds()))):
                # dataset이 index되어 list처럼 동작
                image_id = self.coco.getImgIds(imgIds=index)
                image_infos = self.coco.loadImgs(image_id)[0]

                ann_ids = self.coco.getAnnIds(imgIds=image_infos['id'])
                anns = self.coco.loadAnns(ann_ids)

                # Load the categories in a variable
                cat_ids = self.coco.getCatIds()
                cats = self.coco.loadCats(cat_ids)

                # masks : size가 (height x width)인 2D
                # 각각의 pixel 값에는 "category id + 1" 할당
                # Background = 0
                masks = np.zeros((image_infos["height"], image_infos["width"]))
                # Unknown = 1, General trash = 2, ... , Cigarette = 11
                for i in range(len(anns)):
                    className = get_classname(anns[i]['category_id'], cats)
                    pixel_value = category_names.index(className)
                    masks = np.maximum(self.coco.annToMask(anns[i])*pixel_value, masks)
                masks = masks.astype(np.float32)
                if 10.0 in list(np.unique(masks)): self.mask_label.append(0)
                elif 1.0 in list(np.unique(masks)): self.mask_label.append(1)
                elif 11.0 in list(np.unique(masks)): self.mask_label.append(2)
                elif 6.0 in list(np.unique(masks)): self.mask_label.append(3)
                elif 5.0 in list(np.unique(masks)): self.mask_label.append(4)
                elif 4.0 in list(np.unique(masks)): self.mask_label.append(5)
                elif 8.0 in list(np.unique(masks)): self.mask_label.append(6)
                elif 7.0 in list(np.unique(masks)): self.mask_label.append(7)
                elif 2.0 in list(np.unique(masks)): self.mask_label.append(8)
                elif 3.0 in list(np.unique(masks)): self.mask_label.append(9)
                elif 9.0 in list(np.unique(masks)): self.mask_label.append(10)
                else: self.mask_label.append(11)  
                self.mask_image.append(masks)

          
        
    def __getitem__(self, index: int):
        # dataset이 index되어 list처럼 동작
        image_id = self.coco.getImgIds(imgIds=index)
        image_infos = self.coco.loadImgs(image_id)[0]
        
        # cv2 를 활용하여 image 불러오기
        images = cv2.imread(os.path.join(dataset_path, image_infos['file_name']))
        images = cv2.cvtColor(images, cv2.COLOR_BGR2RGB).astype(np.float32)
        
        if (self.mode in ('train', 'val')):
            # transform -> albumentations 라이브러리 활용
            masks=self.mask_image[index]
            weak_label=self.mask_label[index]
            
            if self.transform is not None:
                transformed = self.transform(image=images, mask=masks)
                images = transformed["image"]
                masks = transformed["mask"]
            
            return images, masks, weak_label, image_infos
        
        if self.mode == 'test':
            # transform -> albumentations 라이브러리 활용
            if self.transform is not None:
                transformed = self.transform(image=images)
                images = transformed["image"]
            
            return images, image_infos
    
    
    def __len__(self) -> int:
        # 전체 dataset의 size를 return
        return len(self.coco.getImgIds())

## Dataset 정의 및 DataLoader 할당

In [7]:
# train.json / validation.json / test.json 디렉토리 설정
train_path = dataset_path + '/train_all.json'
#val_path = dataset_path + '/val.json'
test_path = dataset_path + '/test.json'

# collate_fn needs for batch
def collate_fn(batch):
    return tuple(zip(*batch))

train_transform = A.Compose([
                            A.Rotate(border_mode=1, p=0.5),
                            A.ShiftScaleRotate(border_mode=1, p=0.5),
                            A.HorizontalFlip(p=0.5),
                            A.VerticalFlip(p=0.5),
                            A.Cutout(),
                            A.Normalize(
                                mean=(0.485, 0.456, 0.406),
                                std=(0.229, 0.224, 0.225), max_pixel_value=255.0, p=1.0
                            ),
                            ToTensorV2(),
                            ])

test_transform = A.Compose([
                            A.Normalize(
                                mean=(0.485, 0.456, 0.406),
                                std=(0.229, 0.224, 0.225), max_pixel_value=255.0, p=1.0
                            ),
                            ToTensorV2(),
                           ])

# create own Dataset 1 (skip)
# validation set을 직접 나누고 싶은 경우
# random_split 사용하여 data set을 8:2 로 분할
# train_size = int(0.8*len(dataset))
# val_size = int(len(dataset)-train_size)
# dataset = CustomDataLoader(data_dir=train_path, mode='train', transform=transform)
# train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# create own Dataset 2
# train dataset
train_dataset = CustomDataLoader(data_dir=train_path, mode='train', transform=train_transform)

# test dataset
test_dataset = CustomDataLoader(data_dir=test_path, mode='test', transform=test_transform)

loading annotations into memory...
Done (t=5.32s)
creating index...
index created!
train : Preprocessing...


  0%|          | 0/3272 [00:00<?, ?it/s]

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


## Class Score

In [8]:
def class_score(avrg_class_IoU):
    # Class Score
    class_name=['BG','UNK','General Trash','Paper','Paper pack','Metal','Glass','Plastic','Styrofoam','Plastic Bag','Battery','Clothing']
    print('-'*80)
    print('Validation Class Pred mIoU Score')
    for idx, class_score in enumerate(avrg_class_IoU):
        print('[{}] mIoU : [{:.4f}]'.format(class_name[idx],class_score))
    print('-'*80) 

## CutMix

In [9]:
def rand_bbox(W, H, lam):
    cut_rat = torch.sqrt(1.0 - lam)
    cut_w = (W * cut_rat).type(torch.long)
    cut_h = (H * cut_rat).type(torch.long)
    # uniform
    cx = torch.randint(W, (1,)).to(device)
    cy = torch.randint(H, (1,)).to(device)
    x1 = torch.clamp(cx - cut_w // 2, 0, W)
    y1 = torch.clamp(cy - cut_h // 2, 0, H)
    x2 = torch.clamp(cx + cut_w // 2, 0, W)
    y2 = torch.clamp(cy + cut_h // 2, 0, H)
    return x1, y1, x2, y2

In [10]:
def cutmix_data(x, y, alpha=1.0, p=0.5):
    # x-> img 
    # y-> mask
    if np.random.random() > p:
        return x, y
    W, H = x.size(2), x.size(3)
    shuffle = torch.randperm(x.size(0)).to(device)
    cutmix_x = x
    cutmix_y = y

    lam = torch.distributions.beta.Beta(alpha, alpha).sample().to(device)
    x1, y1, x2, y2 = rand_bbox(W, H, lam)
    cutmix_x[:, :, x1:x2, y1:y2] = x[shuffle, :, x1:x2, y1:y2]
    cutmix_y[:, x1:x2, y1:y2] = y[shuffle, x1:x2, y1:y2]
    # Adjust lambda to match pixel ratio
    #lam = 1 - ((x2 - x1) * (y2 - y1) / float(W * H)).item()
    return cutmix_x, cutmix_y

## train, validation, test 함수 정의

In [11]:
def train(fold, num_epochs, model, data_loader, val_loader, criterion, optimizer, scheduler, device):
    print('-'*80)
    print(f'Fold : [{fold}] Start training..')
    print('-'*80)
    early_stop=EarlyStopping(patience=5,path='./saved/'+str(fold)+'_checkpoint.pt')

    for epoch in range(num_epochs):
        train_loss=[]
        model.train()
        for step, (images, masks, _, _) in enumerate(tqdm(data_loader)):
            images = torch.stack(images)       # (batch, channel, height, width)
            masks = torch.stack(masks).long()  # (batch, channel, height, width)

            # gpu 연산을 위해 device 할당
            images, masks = images.to(device), masks.to(device)

            #Cut-Mix
            images, masks = cutmix_data(images, masks)

            # inference
            outputs = model(images)

            # loss 계산
            loss = criterion(outputs, masks)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss.append(loss.item())

            #Cosine Annealing Scheduler
            #scheduler.step(epoch + step / len(data_loader))

        # validation 주기에 따른 loss 출력 및 best model 저장
        avrg_loss, avrg_IoU, avrg_class_IoU, pb_IoU = validation(model, val_loader, device)
        print('Epoch [{}/{}], Train Loss: {:.4f} Vali Loss: {:.4f}, Vali mIoU: {:.4f}, Vali Public mIoU: {:.4f}'.format(epoch+1, num_epochs, np.mean(train_loss), avrg_loss, avrg_IoU, pb_IoU))
        # Reduce Scheduler
        scheduler.step(avrg_IoU)
        # Class Score
        class_score(avrg_class_IoU)
        # Save
        early_stop(avrg_IoU,model)
        if early_stop.early_stop:
            print('Stop Training.....')
            break

In [12]:
def validation(model, data_loader, device):
    criterion = nn.CrossEntropyLoss().to(device)
    n_class=12
    hist = np.zeros((n_class, n_class))
    print('Start validation')
    model.eval()
    with torch.no_grad():
        total_loss = 0
        cnt = 0
        pb_mIoU_list = []
        for step, (images, masks, _, _) in enumerate(tqdm(data_loader)):
            images = torch.stack(images)       # (batch, channel, height, width)
            masks = torch.stack(masks).long()  # (batch, channel, height, width)

            images, masks = images.to(device), masks.to(device)        

            outputs = model(images)
            loss = criterion(outputs, masks)
            total_loss += loss
            cnt += 1
            
            outputs = torch.argmax(outputs.squeeze(), dim=1).detach().cpu().numpy()
            
            hist = add_hist(hist, masks.detach().cpu().numpy(), outputs, n_class=n_class)
            pb_mIoU = public_label_accuracy_score(masks.detach().cpu().numpy(), outputs, n_class=12)
            pb_mIoU_list.append(pb_mIoU)
        
        vali_mIoU, vali_class_mIoU = label_accuracy_score(hist)
        avrg_loss = total_loss / cnt

    return avrg_loss, vali_mIoU, vali_class_mIoU, np.mean(pb_mIoU_list)

## 모델 생성 및 Loss function, Optimizer 정의

In [13]:
# Optimizer 정의
#optimizer = torch.optim.Adam(params = model.parameters(), lr = 1.2e-7)

# Scheduler 정의
#scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=1,threshold_mode='abs',min_lr=1e-8, verbose=True)
#scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, eta_min=1.2e-7)
#scheduler = CosineAnnealingWarmUpRestarts(optimizer, T_0=20, eta_max=learning_rate,  T_up=2, gamma=0.5)

In [14]:
folds=StratifiedKFold(n_splits=5,shuffle=True)
for current_fold,(train_idx, vali_idx) in enumerate(folds.split(train_dataset,train_dataset.mask_label)):
    model = smp.DeepLabV3Plus('resnext50_32x4d', encoder_weights="swsl", classes=12)
    model.eval()
    model.to(device)

    train_data=torch.utils.data.Subset(train_dataset,train_idx)
    vali_data=torch.utils.data.Subset(train_dataset,vali_idx)

    train_loader=DataLoader(train_data,batch_size=batch_size,shuffle=True,collate_fn=collate_fn,num_workers=0)
    val_loader=DataLoader(vali_data,batch_size=batch_size,shuffle=False,collate_fn=collate_fn,num_workers=0)

    criterion = nn.CrossEntropyLoss().to(device)
    optimizer = torch.optim.Adam(params = model.parameters(), lr = learning_rate)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=1,threshold_mode='abs',min_lr=1e-8, verbose=True)
    #scheduler = CosineAnnealingWarmUpRestarts(optimizer, T_0=20, eta_max=learning_rate,  T_up=2, gamma=0.5)
    
    train(current_fold, num_epochs, model, train_loader, val_loader, criterion, optimizer, scheduler, device)

--------------------------------------------------------------------------------
Fold : [0] Start training..
--------------------------------------------------------------------------------


  0%|          | 0/131 [00:00<?, ?it/s]

Start validation


  0%|          | 0/33 [00:00<?, ?it/s]

Epoch [1/40], Train Loss: 0.9311 Vali Loss: 0.5907, Vali mIoU: 0.2645, Vali Public mIoU: 0.3093
--------------------------------------------------------------------------------
Validation Class Pred mIoU Score
[BG] mIoU : [0.9120]
[UNK] mIoU : [0.0000]
[General Trash] mIoU : [0.2362]
[Paper] mIoU : [0.6265]
[Paper pack] mIoU : [0.0000]
[Metal] mIoU : [0.0653]
[Glass] mIoU : [0.0262]
[Plastic] mIoU : [0.2771]
[Styrofoam] mIoU : [0.3736]
[Plastic Bag] mIoU : [0.6571]
[Battery] mIoU : [0.0000]
[Clothing] mIoU : [0.0000]
--------------------------------------------------------------------------------


...

Epoch [27/40], Train Loss: 0.0956 Vali Loss: 0.2808, Vali mIoU: 0.5787, Vali Public mIoU: 0.4912
Epoch    27: reducing learning rate of group 0 to 3.1250e-06.
--------------------------------------------------------------------------------
Validation Class Pred mIoU Score
[BG] mIoU : [0.9508]
[UNK] mIoU : [0.0000]
[General Trash] mIoU : [0.4851]
[Paper] mIoU : [0.7558]
[Paper pack] mIoU : [0.5072]
[Metal] mIoU : [0.5054]
[Glass] mIoU : [0.5077]
[Plastic] mIoU : [0.4543]
[Styrofoam] mIoU : [0.7264]
[Plastic Bag] mIoU : [0.8205]
[Battery] mIoU : [0.6970]
[Clothing] mIoU : [0.5345]
--------------------------------------------------------------------------------


  0%|          | 0/131 [00:00<?, ?it/s]

Start validation


  0%|          | 0/33 [00:00<?, ?it/s]

Epoch [28/40], Train Loss: 0.0956 Vali Loss: 0.2744, Vali mIoU: 0.5747, Vali Public mIoU: 0.4732
--------------------------------------------------------------------------------
Validation Class Pred mIoU Score
[BG] mIoU : [0.9532]
[UNK] mIoU : [0.0000]
[General Trash] mIoU : [0.4763]
[Paper] mIoU : [0.7573]
[Paper pack] mIoU : [0.4422]
[Metal] mIoU : [0.5148]
[Glass] mIoU : [0.4757]
[Plastic] mIoU : [0.4516]
[Styrofoam] mIoU : [0.7260]
[Plastic Bag] mIoU : [0.8262]
[Battery] mIoU : [0.6732]
[Clothing] mIoU : [0.6000]
--------------------------------------------------------------------------------
Stop Training.....


## 저장된 model 불러오기 (학습된 이후) 

In [19]:
# best model 저장된 경로
model_path = './saved/15_checkpoint.pt'

# best model 불러오기
checkpoint = torch.load(model_path, map_location=device)
model.load_state_dict(checkpoint)

# 추론을 실행하기 전에는 반드시 설정 (batch normalization, dropout 를 평가 모드로 설정)
# model.eval()

<All keys matched successfully>

## submission을 위한 test 함수 정의

In [ ]:
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          num_workers=1,
                                          collate_fn=collate_fn)

In [20]:
def test(model, data_loader, device):
    model.to(device)
    size = 256
    transform = A.Compose([A.Resize(256, 256)])
    print('Start prediction.')
    model.eval()
    
    file_name_list = []
    preds_array = np.empty((0, size*size), dtype=np.long)
    
    with torch.no_grad():
        for step, (imgs, image_infos) in enumerate(tqdm(test_loader)):

            # inference (512 x 512)
            outs = model(torch.stack(imgs).to(device))
            oms = torch.argmax(outs.squeeze(), dim=1).detach().cpu().numpy()
            
            # resize (256 x 256)
            temp_mask = []
            for img, mask in zip(np.stack(imgs), oms):
                transformed = transform(image=img, mask=mask)
                mask = transformed['mask']
                temp_mask.append(mask)

            oms = np.array(temp_mask)
            
            oms = oms.reshape([oms.shape[0], size*size]).astype(int)
            preds_array = np.vstack((preds_array, oms))
            
            file_name_list.append([i['file_name'] for i in image_infos])
    print("End prediction.")
    file_names = [y for x in file_name_list for y in x]
    
    return file_names, preds_array

## submission.csv 생성

In [21]:
# sample_submisson.csv 열기
submission = pd.read_csv('./submission/sample_submission.csv', index_col=None)

# test set에 대한 prediction
file_names, preds = test(model, test_loader, device)

# PredictionString 대입
for file_name, string in zip(file_names, preds):
    submission = submission.append({"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in string.tolist())}, 
                                   ignore_index=True)

# submission.csv로 저장
submission.to_csv("./submission/deeplabv3plus_maxIoU.csv", index=False)

Start prediction.


  0%|          | 0/42 [00:00<?, ?it/s]

End prediction.


## Reference

